# Constructing time-varying and time-invariant control variables 

Welcome to the next chapter of our exploration into the factors that shape the success of movie directors! Our goal is to uncover how connections with network brokers—those key individuals who link different creative circles—impact the careers of male and female movie directors differently.

**What are we aiming to do?**

To accurately analyze how the social capital of collaborators influences a director's career, we need to account for various factors that might also affect the results. In this notebook, we'll create several control variables, including directors' past success, the genres they specialize in, their entry time into the industry, their pre-directing film experience, and industry trends and market competition during their careers.

Control variables allow us to compare directors under similar conditions, ensuring that any observed effects on career success are due to collaborators' social capital rather than other factors like debut year or genre focus. By controlling for these factors, we can isolate the true impact of social networks on career trajectories.

**Our Approach:**

1. Creating Time-Invariant Control Variables for Cross-Sectional Data

Our cross-sectional data summarizes directors' careers during their first decade in the industry. Here, we need to control for factors might influence both the social capital of collaborators at the directors' debut and their career outcomes ten years later. These factors include:
- Directors' prior experience in non-directing roles (e.g., as writers, producers, actors) before their directorial debut.
- Collaborators' past success before the directors' debut (the average ratings and number of votes of the movies colaborators worked on in the three years before they worked with the director).
- The year of the directors' debut, capturing the context of the time when they started.

2. Creating Time-Varying and Time-Invariant Control Variables for Time-Series Data

Our time-series data tracks directors' careers year by year from debut until 2023. We need to account for factors that remain the same as well as those that change throughout their careers. 

We create time-varying control variables, including:
- Directors' annual success metrics tracking the average ratings and number of votes of the movies directed in the previous three years.
- Directors' genre specialization, tracking the genres of the movies directed in the previous three years.
- Industry conditions at each career point.



**Looking Ahead**

By the end of this notebook, we will have a set of control variables that account for both time-varying and time-invariant factors. This will allow us to make more accurate comparisons and better understand the true impact of collaborators' social capital on the career success of movie directors.

# 1. Load and inspect data 

To create control variables, we use three key datasets created in previous phases:
- `directors_years`, which traces the directors' career from their debut until 2023, with each row representing a year in their career.
- `directors_cross_sectional`, which summarizes each director's career 10 years following their debut, with each row representing one director.
- `directors_full_filmography`, which lists the movies directed by the directors in our sample, each row linking a movie to a director.

In [1]:
# Importing necessary libraries for data manipulation and handling
import pandas as pd  # data manipulation
import numpy as np # numerical operations
import os  # interacting with the operating system, such as changing directories
import ast  # evaluating string literals as Python expressions

# Set the working directory to where the project files are located
os.chdir('/Users/mac/Library/CloudStorage/OneDrive-McGillUniversity/Work/Projects/Gender and brokerage/WomenLeaders_SocialNetworks')

In [2]:
# load yearly data of directors
directors_years = pd.read_csv('directors_years.csv')

# Display the first few rows of the dataset to understand their structure
directors_years.head()

,nconst_director,year,debut_year
0,nm1131265,2004,2004
1,nm1131265,2005,2004
2,nm1131265,2006,2004
3,nm1131265,2007,2004
4,nm1131265,2008,2004


In [3]:
# load cross-sectional data of directors
directors_cross_sectional = pd.read_csv('directors_cross_sectional.csv')

# Display the first few rows of the dataset to understand their structure
directors_cross_sectional.head()

,nconst_director,debut_year,time_to_second_movie,event,total_movies
0,nm1131265,2004,9.0,1,1.0
1,nm1130611,2004,11.0,0,0.0
2,nm0290651,2003,11.0,0,0.0
3,nm0404033,2003,5.0,1,1.0
4,nm0000417,2005,2.0,1,1.0


In [4]:
# Load data on full filmography of directors
directors_full_filmography = pd.read_csv('directors_full_filmography.csv')

# Display the first few rows of the dataset to understand their structure
directors_full_filmography.head()

,tconst,startYear,genres,nconst,firstYear,averageRating,numVotes
0,tt0108549,2004.0,"Comedy,Mystery",nm1131265,2004.0,7.8,34.0
1,tt0108549,2004.0,"Comedy,Mystery",nm1130611,2004.0,7.8,34.0
2,tt0117461,2003.0,"Comedy,Romance",nm0290651,2003.0,6.3,24.0
3,tt0117743,2008.0,"Drama,Romance",nm0404033,2003.0,6.7,64.0
4,tt0118141,2005.0,Drama,nm0000417,2005.0,5.3,950.0


In [5]:
directors_full_filmography = directors_full_filmography.rename(columns={'nconst': 'nconst_director'})

# 2. Creating time-invariant control variables for cross-sectional data

For cross-sectional data, we need to control for any factor that might influence both the social capital of collaborators at the directors' debut and the directors' career outcomes ten years later. We'll look at the following factors: directors' prior experience in nondirecting roles (e.g., as a writer, producer, actor, etc.) before their directorial debut, collaborators' past success before the directors' debut, and the directors' debut year.

## 2.1. Directors' prior filmmaking experience 

Directors with prior experience in the film industry might have better networks and industry knowledge, influencing both their choice of collaborators and their career outcomes.

In [6]:
directors_pre_debut_careers = pd.read_csv('directors_pre_debut_careers.csv')
directors_pre_debut_careers.head()

,nconst,had_prior_experience,movie_count,roles
0,nm1131265,0,0.0,NaN
1,nm1130611,0,0.0,NaN
2,nm0290651,0,0.0,NaN
3,nm0404033,0,0.0,NaN
4,nm0000417,1,14.0,"['archive_footage', 'actor']"


This dataset contains several columns, including a unique identifier for each director (`nconst`), an indicator variable for whether they had prior experience in non-directing roles before becoming a director (`had_prior_experience`), the number of movies they were involved in before their directorial debut (`movie_count`), and the role they took on in these movies (`roles`).

Let's create dummy variables for each role that directors took on before they became directors. Since the roles are stored as string representations of lists, we first need to convert these strings back into lists, then create a column for each unique role across all these lists. 

In [7]:
# Function to safely convert string representations of lists into actual lists
def convert_to_list(row):
    try:
        return ast.literal_eval(row) if isinstance(row, str) else []
    except ValueError:
        return []

# Apply the function to the 'roles' column
directors_pre_debut_careers['roles'] = directors_pre_debut_careers['roles'].apply(convert_to_list)

# Use pandas' get_dummies to create dummy variables for each role
# First, explode the 'roles' column to have a separate row for each role per director
df_directors_exploded = directors_pre_debut_careers.explode('roles')

# Now, create dummy variables
dummy_roles = pd.get_dummies(df_directors_exploded['roles'], prefix='pre_debut_role', drop_first=True)

# Since we exploded the DataFrame, multiple rows now represent the same director.
# We need to group by 'nconst' and sum the dummies to collapse these back together.
df_roles_dummies = dummy_roles.groupby(df_directors_exploded['nconst']).sum().reset_index()

df_roles_dummies.head()

,nconst,pre_debut_role_actress,pre_debut_role_archive_footage,pre_debut_role_cinematographer,pre_debut_role_composer,pre_debut_role_editor,pre_debut_role_producer,pre_debut_role_production_designer,pre_debut_role_writer
0,nm0000083,0,0,0,0,1,0,0,1
1,nm0000136,0,1,0,0,0,0,0,0
2,nm0000147,0,0,0,0,0,0,0,0
3,nm0000154,0,0,0,0,0,0,0,0
4,nm0000155,1,0,0,0,0,0,0,0


In [8]:
directors_cross_sectional.head()

,nconst_director,debut_year,time_to_second_movie,event,total_movies
0,nm1131265,2004,9.0,1,1.0
1,nm1130611,2004,11.0,0,0.0
2,nm0290651,2003,11.0,0,0.0
3,nm0404033,2003,5.0,1,1.0
4,nm0000417,2005,2.0,1,1.0


In [9]:
directors_cross_sectional_control_variables = pd.merge(directors_cross_sectional[['nconst_director']],
                                                       directors_pre_debut_careers[['nconst', 'had_prior_experience', 'movie_count']].rename(columns={'movie_count': 'pre_debut_movie_count', 'nconst': 'nconst_director'}),
                                                       on=['nconst_director'], how='left').merge(
                                                           df_roles_dummies.rename(columns={'nconst': 'nconst_director'}),
                                                           on=['nconst_director'], how='left')

## 2.2. Collaborators' past success

The reputation or success of the projects the collaborators' were involved in could influence their social capital at the directors's debut and the director's success after debut. We use average ratings and total number of votes on IMDb for collaborators' past movies as indicators of collaborators' past success.

To compute these metrics, we identify the collaborators a director worked with for their debut movie and identify the movies these collaborators worked on three years prior. Then, we merge this data with IMDb's `title.ratings` data to include the ratings and votes information. Finally we calculate the average rating and total number of votes for the movies of the collaborators, which reflect their past success. 

In [10]:
collaborators_by_year = pd.read_pickle("collaborators_by_year.pkl")
collaborators_by_year.head()

,nconst_director,year,nconst_collaborator,debut_year,brokerage_scores,num_collaborators,avg_constraint,max_constraint,avg_local_clustering,max_local_clustering,avg_reverse_constraint,max_reverse_constraint,avg_effective_size,max_effective_size,avg_reverse_local_clustering,max_reverse_local_clustering,avg_degree_centrality,max_degree_centrality
0,nm0000083,2008,"[nm0613273, nm0690875, nm0917916, nm1677271, n...",2007,[{'nm0613273': {'constraint': 0.51412742382271...,5.0,0.394400,0.514127,0.730994,1.0,0.605610,0.874041,4.842105,15.210526,0.269016,0.789484,8.800000,19.0
1,nm0000083,2009,"[nm0613273, nm0690875, nm0917916, nm1677271, n...",2007,[{'nm0613273': {'constraint': 0.52367346938775...,5.0,0.408532,0.523673,0.746032,1.0,0.591478,0.834517,3.857143,10.285714,0.253978,0.714296,7.800000,14.0
2,nm0000083,2010,"[nm0613273, nm0690875, nm0917916, nm1677271, n...",2007,[{'nm0613273': {'constraint': 0.42296933569346...,7.0,0.433951,0.513961,0.760204,1.0,0.566059,0.683205,2.596939,5.000000,0.239806,0.571439,6.285714,8.0
3,nm0000083,2011,"[nm0613273, nm1677271, nm0662263, nm1730786]",2007,"[{'nm0613273': {'constraint': 0.6865234375, 'e...",4.0,0.606336,0.686523,0.857143,1.0,0.393674,0.634238,2.000000,5.000000,0.142867,0.571439,5.000000,8.0
4,nm0000083,2012,"[nm0613273, nm1677271, nm0662263, nm1730786]",2007,"[{'nm0613273': {'constraint': 0.6865234375, 'e...",4.0,0.610596,0.686523,0.857143,1.0,0.389414,0.617198,2.000000,5.000000,0.142867,0.571439,5.000000,8.0


In [11]:
movies_collaborations_2000_2023 = pd.read_csv("movies_collaborations_2000_2023.csv")
movies_collaborations_2000_2023.head()

,tconst,nconst,category,startYear
0,tt0011801,nm0483944,writer,2019.0
1,tt0011801,nm1773808,cinematographer,2019.0
2,tt0013274,nm0412842,director,2021.0
3,tt0013274,nm0895048,director,2021.0
4,tt0013274,nm13054604,producer,2021.0


In [12]:
# Filter rows for each director in the year after their debut year
# because of the lagged data structure, these are the people the directors worked with for their debut movie
collaborators_at_debut = collaborators_by_year[collaborators_by_year['year'] == collaborators_by_year['debut_year'] + 1]

# Collect all necessary collaborator IDs and their respective target years
collaborators_target_years = []
for index, row in collaborators_at_debut.iterrows():
    year = row['year']
    year_min = year - 3 # Set the start of the timeframe to three years prior.
    year_max = year - 1 # Set the end of the timeframe to a year prior.
    collaborators = row['nconst_collaborator']
    for collaborator in collaborators:
        collaborators_target_years.append((collaborator, year_min, year_max, row['nconst_director'], year))

# Create a DataFrame from the collected data
collaborators_target_years = pd.DataFrame(collaborators_target_years, columns=['nconst', 'year_min', 'year_max', 'nconst_director', 'year'])

collaborators_target_years = pd.merge(collaborators_target_years, 
                                      movies_collaborations_2000_2023.drop(columns=['category']), 
                                      on='nconst', how='left')

collaborators_target_years = collaborators_target_years[
    (collaborators_target_years['startYear'] >= collaborators_target_years['year_min']) &
    (collaborators_target_years['startYear'] <= collaborators_target_years['year_max'])
]

collaborators_target_years = collaborators_target_years[['nconst_director', 'year', 'tconst']].drop_duplicates()

In [13]:
# Integrating IMDb Ratings
# Accessing and merging IMDb's 'title.ratings' dataset enhances our filmography data with public reception metrics.
# This inclusion of ratings and votes offers an additional layer of analysis for evaluating directors' works.
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
movies_ratings = pd.read_csv(url_ratings, sep='\t', compression='gzip')

# Displaying the first few rows for an overview of the data
movies_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2059
1,tt0000002,5.6,277
2,tt0000003,6.5,2017
3,tt0000004,5.4,180
4,tt0000005,6.2,2783


In [14]:
collaborators_movies_ratings = collaborators_target_years.merge(movies_ratings, on='tconst', how='left')
collaborators_movies_ratings.head(10)

,nconst_director,year,tconst,averageRating,numVotes
0,nm0000083,2008,tt0969216,6.4,116.0
1,nm0000083,2008,tt0762149,NaN,NaN
2,nm0000083,2008,tt0350374,6.2,260.0
3,nm0000083,2008,tt0433686,6.8,1568.0
4,nm0000083,2008,tt0825749,6.6,166.0
5,nm0000136,2010,tt8045334,8.4,80.0
6,nm0000147,2011,tt1733300,6.1,38.0
7,nm0000147,2011,tt1156524,7.3,643.0
8,nm0000154,2005,tt0277434,7.2,151702.0
9,nm0000154,2005,tt0314676,5.4,8960.0


In [15]:
# Group by director and calculate average rating and total votes
collaborators_movies_ratings = collaborators_movies_ratings.groupby('nconst_director').agg(
    average_rating=('averageRating', 'mean'),
    total_votes=('numVotes', 'sum')
).reset_index()

# Set total_votes to NaN if average_rating is NaN
collaborators_movies_ratings['total_votes'] = collaborators_movies_ratings['total_votes'].where(collaborators_movies_ratings['average_rating'].notna(), np.nan)

In [16]:
directors_cross_sectional_control_variables = pd.merge(directors_cross_sectional_control_variables,
                                                       collaborators_movies_ratings.rename(
                                                           columns={'average_rating':'collaborator_movie_ratings', 
                                                                    'total_votes':'collaborator_movie_votes'}),
                                                       on=['nconst_director'], how='left')

In [17]:
directors_cross_sectional_control_variables.head()

,nconst_director,had_prior_experience,pre_debut_movie_count,pre_debut_role_actress,pre_debut_role_archive_footage,pre_debut_role_cinematographer,pre_debut_role_composer,pre_debut_role_editor,pre_debut_role_producer,pre_debut_role_production_designer,pre_debut_role_writer,collaborator_movie_ratings,collaborator_movie_votes
0,nm1131265,0,0.0,0,0,0,0,0,0,0,0,7.35,115.0
1,nm1130611,0,0.0,0,0,0,0,0,0,0,0,7.35,115.0
2,nm0290651,0,0.0,0,0,0,0,0,0,0,0,4.95,1432.0
3,nm0404033,0,0.0,0,0,0,0,0,0,0,0,4.96,963.0
4,nm0000417,1,14.0,0,1,0,0,0,0,0,0,5.75,1598.0


## 2.3. Directors' debut year 

Finally, we also control for industry trends and external factors that directors faced when they started their careers, which could impact both the collaborators’ social capital and the director’s long-term success (e.g., economic conditions, technological changes, industry norms).

For this, we create dummy variables for debut year.

In [18]:
debut_year_dummies = pd.get_dummies(directors_cross_sectional.drop(columns=['time_to_second_movie', 'event', 'total_movies']), 
                                    columns=['debut_year'], prefix='debut_year', drop_first =  True)

directors_cross_sectional_control_variables = pd.merge(directors_cross_sectional_control_variables, debut_year_dummies, 
                                                       on=['nconst_director'], how='left')

Finally, we convert debut year dummy columns from boolean (True/False) to integer (1/0).

In [19]:
# Select columns that start with 'debut_year_' or 'year_'
dummy_columns = [col for col in directors_cross_sectional_control_variables.columns if col.startswith('debut_year_')]

# Convert selected columns to integer
directors_cross_sectional_control_variables[dummy_columns] = directors_cross_sectional_control_variables[dummy_columns].astype(int)

In [20]:
directors_cross_sectional_control_variables.head()

,nconst_director,had_prior_experience,pre_debut_movie_count,pre_debut_role_actress,pre_debut_role_archive_footage,pre_debut_role_cinematographer,pre_debut_role_composer,pre_debut_role_editor,pre_debut_role_producer,pre_debut_role_production_designer,...,debut_year_2004,debut_year_2005,debut_year_2006,debut_year_2007,debut_year_2008,debut_year_2009,debut_year_2010,debut_year_2011,debut_year_2012,debut_year_2013
0,nm1131265,0,0.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,nm1130611,0,0.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,nm0290651,0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,nm0404033,0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,nm0000417,1,14.0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [21]:
# save the data for future analysis 
# directors_years_control_variables.to_csv('directors_years_control_variables.csv', index=False)
directors_cross_sectional_control_variables.to_csv('directors_cross_sectional_control_variables.csv', index=False)

# 3. Creating time-varying control variables for time-series data

For our panel data, we create time-varying control variables including directors' success, collaborators' success, directors' genre specialization, industry conditions at each point in their career. We'll also incorporate the time-invariant control variables calculated above.

## 3.1. Directors' past success

We calculate metrics indicating directors' past success for each year, considering data 3 years before the focal year:
- Total Movies: The number of movies directed.
- Average Rating: The average rating of these movies.
- Total Votes: The total number of votes these movies received.

In [22]:
directors_full_filmography['startYear'] = directors_full_filmography['startYear'].astype(int)

# Initialize a list to store the computed metrics for each year
past_success_metrics_list = []

# Calculate metrics for each year, considering data up to the year before the focal year
for year in range(2003, directors_full_filmography['startYear'].max() + 1):
    # Filter movies released in the 3 years prior to the focal year
    filtered_df = directors_full_filmography[
        (directors_full_filmography['startYear'] < year) & 
        (directors_full_filmography['startYear'] >= year - 3)
    ]
    
    # Compute past success metrics
    metrics_df = filtered_df.groupby('nconst_director').agg(
        total_movies=('tconst', 'nunique'),
        average_rating=('averageRating', 'mean'),
        total_votes=('numVotes', 'sum')
    ).reset_index()

    # Set total_votes to NaN if average_rating is NaN
    metrics_df['total_votes'] = metrics_df['total_votes'].where(metrics_df['average_rating'].notna(), np.nan)
    
    # Add the year to each row in the aggregated DataFrame
    metrics_df['year'] = year
    
    # Append the yearly data to the list
    past_success_metrics_list.append(metrics_df)

# Concatenate all yearly DataFrames to create a single DataFrame with all metrics
past_success_metrics = pd.concat(past_success_metrics_list, ignore_index=True)

# Display the first few ros of the data
past_success_metrics.head()

,nconst_director,total_movies,average_rating,total_votes,year
0,nm0000362,1,5.9,60259.0,2004
1,nm0000410,1,6.5,6238.0,2004
2,nm0000996,1,5.8,1219.0,2004
3,nm0001592,1,5.8,63.0,2004
4,nm0001736,1,4.4,4989.0,2004


In [23]:
# Incorporate past success metrics into time series data tracing directors' career
directors_years_control_variables = pd.merge(directors_years, past_success_metrics, 
                                             on=['nconst_director', 'year'], how='left')

# Rename columns for clarity
directors_years_control_variables = directors_years_control_variables.rename(
    columns={'total_movies':'director_total_movies', 
             'average_rating':'director_raverage_rating',
             'total_votes':'director_total_votes'})

In [24]:
directors_years_control_variables.head()

,nconst_director,year,debut_year,director_total_movies,director_raverage_rating,director_total_votes
0,nm1131265,2004,2004,NaN,NaN,NaN
1,nm1131265,2005,2004,1.0,7.8,34.0
2,nm1131265,2006,2004,1.0,7.8,34.0
3,nm1131265,2007,2004,1.0,7.8,34.0
4,nm1131265,2008,2004,NaN,NaN,NaN


## 3.2. Directors' genre specilization

Next, we analyze directors' genre specialization by examining the genres of movies they directed in the three years prior to each focal year and calculating the proportion of movies in each genre relative to the total number of movies directed. 

In [25]:
# Prepare the data
# Replace '\N' with np.nan in the 'genres' column
directors_full_filmography['genres'] = directors_full_filmography['genres'].replace('\\N', np.nan)

# Assuming genres are separated by commas, split them into lists
directors_full_filmography['genres'] = directors_full_filmography['genres'].str.split(',')

# Explode the DataFrame so each genre gets its own row
directors_filmography_exploded = directors_full_filmography.explode('genres')

In [26]:
# Display the first few rows of the data
directors_filmography_exploded

,tconst,startYear,genres,nconst_director,firstYear,averageRating,numVotes
0,tt0108549,2004,Comedy,nm1131265,2004.0,7.8,34.0
0,tt0108549,2004,Mystery,nm1131265,2004.0,7.8,34.0
1,tt0108549,2004,Comedy,nm1130611,2004.0,7.8,34.0
1,tt0108549,2004,Mystery,nm1130611,2004.0,7.8,34.0
2,tt0117461,2003,Comedy,nm0290651,2003.0,6.3,24.0
...,...,...,...,...,...,...,...
128529,tt9916362,2020,History,nm1893148,2008.0,6.4,5687.0
128530,tt9916538,2019,Drama,nm4457074,2011.0,8.6,7.0
128531,tt9916622,2015,Documentary,nm9272490,2012.0,NaN,NaN
128532,tt9916754,2013,Documentary,nm9272490,2012.0,NaN,NaN


In [27]:
# Count the occurrences of each genre
genre_counts = directors_filmography_exploded['genres'].value_counts()

# Filter genres that appear at least 100 times
frequent_genres = genre_counts[genre_counts >= 100].index

# Filter the DataFrame to keep only rows with genres that appear at least 100 times
directors_filmography_exploded = directors_filmography_exploded[directors_filmography_exploded['genres'].isin(frequent_genres)]

# Calculate Genre Counts and Total Movies Made
# Initialize a DataFrame to hold the genre specialization results
genre_specialization = []

# Iterate through each year, up to the last year in the dataset
for year in range(2003, directors_full_filmography['startYear'].max() + 1):
    # Filter movies released in the 3 years prior to the focal year
    yearly_data = directors_filmography_exploded[
        (directors_filmography_exploded['startYear'] < year) & 
        (directors_filmography_exploded['startYear'] >= year - 3)
    ]
    
    # Calculate the number of movies in each genre by each director
    genre_counts = yearly_data.groupby(['nconst_director', 'genres']).size().reset_index(name='genre_count')
    
    # Calculate the total number of movies made by each director
    total_movies = yearly_data.groupby('nconst_director').size().reset_index(name='total_movies')
    
    # Merge genre counts with total movies to calculate specialization
    merged_data = pd.merge(genre_counts, total_movies, on='nconst_director')
    
    # Calculate the genre specialization
    merged_data['specialization'] = merged_data['genre_count'] / merged_data['total_movies']
    
    # Add the focal year to the results
    merged_data['year'] = year
    
    # Append the results for this year to the list
    genre_specialization.append(merged_data)

# Concatenate all yearly results into a single DataFrame
genre_specialization = pd.concat(genre_specialization, ignore_index=True)

# pivot to have a wider format with genres as columns
genre_specialization = genre_specialization.pivot_table(index=['nconst_director', 'year'], columns='genres', values='specialization', fill_value=0).reset_index()

In [28]:
# display the first few rows of the data to check its content and structure
genre_specialization.head()

genres,nconst_director,year,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,nm0000083,2008,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,nm0000083,2009,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,nm0000083,2010,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,nm0000083,2023,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nm0000136,2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Incorporate genre specialization into time series data tracing directors' career
directors_years_control_variables = pd.merge(directors_years_control_variables, genre_specialization, 
                                             on=['nconst_director', 'year'], how='left')

In [30]:
directors_years_control_variables.head(20)

,nconst_director,year,debut_year,director_total_movies,director_raverage_rating,director_total_votes,Action,Adventure,Animation,Biography,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,nm1131265,2004,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nm1131265,2005,2004,1.0,7.8,34.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,nm1131265,2006,2004,1.0,7.8,34.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,nm1131265,2007,2004,1.0,7.8,34.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nm1131265,2008,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,nm1131265,2009,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,nm1131265,2010,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,nm1131265,2011,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,nm1131265,2012,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,nm1131265,2013,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are missing data in several places. The first is during their debut year and inactive periods, when they have not released a movie in the previous three years. We'll recode these missing values to zero to reflect inactivity. There can also be missing data in `average_rating`, `total_votes`, or genres columns but not in `total_movies`, indicating that there is no info on ratings, votes, or genres of the movies directed. We'll leave these cases as is to reflect the absence of data.

In [31]:
# Identify the columns from the 4th column onward
start_col = 3  # Adjust this index based on your actual DataFrame structure
columns_to_check = directors_years_control_variables.columns[start_col:]

# Create a mask to identify rows where all specified columns are NaN
mask = directors_years_control_variables[columns_to_check].isna().all(axis=1)

# Apply the recoding using the mask
directors_years_control_variables.loc[mask, columns_to_check] = 0

# Display the results
directors_years_control_variables.head(20)

,nconst_director,year,debut_year,director_total_movies,director_raverage_rating,director_total_votes,Action,Adventure,Animation,Biography,...,Music,Musical,Mystery,News,Romance,Sci-Fi,Sport,Thriller,War,Western
0,nm1131265,2004,2004,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,nm1131265,2005,2004,1.0,7.8,34.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,nm1131265,2006,2004,1.0,7.8,34.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,nm1131265,2007,2004,1.0,7.8,34.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nm1131265,2008,2004,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,nm1131265,2009,2004,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,nm1131265,2010,2004,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,nm1131265,2011,2004,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,nm1131265,2012,2004,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,nm1131265,2013,2004,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3.3. Collaborators' past success

Similar to the directors, we also calculate metrics indicating collaborators' past success for each year, considering data 3 years before the focal year:
- Total Movies: The number of movies collaborators worked on.
- Average Rating: The average rating of these movies.
- Total Votes: The total number of votes these movies received.

In [32]:
# Collect all necessary collaborator IDs and their respective target years
collaborators_target_years = []
for index, row in collaborators_by_year.iterrows():
    year = row['year']
    year_min = year - 3 # Set the start of the timeframe to three years prior.
    year_max = year - 1 # Set the end of the timeframe to a year prior.
    collaborators = row['nconst_collaborator']
    for collaborator in collaborators:
        collaborators_target_years.append((collaborator, year_min, year_max, row['nconst_director'], year))

# Create a DataFrame from the collected data
collaborators_target_years = pd.DataFrame(collaborators_target_years, columns=['nconst', 'year_min', 'year_max', 'nconst_director', 'year'])

collaborators_target_years = pd.merge(collaborators_target_years, 
                                      movies_collaborations_2000_2023.drop(columns=['category']), 
                                      on='nconst', how='left')

collaborators_target_years = collaborators_target_years[
    (collaborators_target_years['startYear'] >= collaborators_target_years['year_min']) &
    (collaborators_target_years['startYear'] <= collaborators_target_years['year_max'])
]

collaborators_target_years = collaborators_target_years[['nconst_director', 'year', 'tconst']].drop_duplicates()

In [33]:
collaborators_movies_ratings = collaborators_target_years.merge(movies_ratings, on='tconst', how='left')
collaborators_movies_ratings.head(10)

,nconst_director,year,tconst,averageRating,numVotes
0,nm0000083,2008,tt0969216,6.4,116.0
1,nm0000083,2008,tt0762149,NaN,NaN
2,nm0000083,2008,tt0350374,6.2,260.0
3,nm0000083,2008,tt0433686,6.8,1568.0
4,nm0000083,2008,tt0825749,6.6,166.0
5,nm0000083,2009,tt0969216,6.4,116.0
6,nm0000083,2009,tt0762149,NaN,NaN
7,nm0000083,2009,tt0433686,6.8,1568.0
8,nm0000083,2009,tt0825749,6.6,166.0
9,nm0000083,2010,tt0969216,6.4,116.0


In [34]:
# Group by director and year and calculate average rating and total votes of collaborators' movies
collaborators_movies_ratings = collaborators_movies_ratings.groupby(['nconst_director', 'year']).agg(
    total_movies=('tconst', 'nunique'),
    average_rating=('averageRating', 'mean'),
    total_votes=('numVotes', 'sum')
).reset_index()

# Set total_votes to NaN if average_rating is NaN
collaborators_movies_ratings['total_votes'] = collaborators_movies_ratings['total_votes'].where(collaborators_movies_ratings['average_rating'].notna(), np.nan)

In [36]:
directors_years_control_variables = pd.merge(directors_years_control_variables,
                                             collaborators_movies_ratings.rename(
                                                 columns={'total_movies': 'collaborator_total_movies',
                                                          'average_rating':'collaborator_movie_ratings', 
                                                          'total_votes':'collaborator_movie_votes'}),
                                             on=['nconst_director', 'year'], how='left')

In [40]:
directors_years_control_variables.head(10)

,nconst_director,year,debut_year,director_total_movies,director_raverage_rating,director_total_votes,Action,Adventure,Animation,Biography,...,News,Romance,Sci-Fi,Sport,Thriller,War,Western,collaborator_total_movies,collaborator_movie_ratings,collaborator_movie_votes
0,nm1131265,2004,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,nm1131265,2005,2004,1.0,7.8,34.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.35,115.0
2,nm1131265,2006,2004,1.0,7.8,34.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.35,115.0
3,nm1131265,2007,2004,1.0,7.8,34.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,7.30,101.0
4,nm1131265,2008,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
5,nm1131265,2009,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.90,674.0
6,nm1131265,2010,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.90,674.0
7,nm1131265,2011,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.90,674.0
8,nm1131265,2012,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
9,nm1131265,2013,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [41]:
# List of columns to fill NA values conditionally
columns_to_fill_na = ['collaborator_total_movies', 'collaborator_movie_ratings', 'collaborator_movie_votes']

# Create a mask where for directors' inactive years (director_total_movies is 0)
mask = (directors_years_control_variables['director_total_movies'] == 0)

# Fill NA values in the specified columns where the mask is True
for column in columns_to_fill_na:
    directors_years_control_variables.loc[mask, column] = directors_years_control_variables.loc[mask, column].fillna(0)

## 3.4. Industry trends and market competition

Next, we include year dummies to control for time-specific effects. Including a dummy for every category would result in perfect multicollinearity because the sum of all dummy variables would always equal one, making it impossible to estimate the model parameters. Thus, we'll leave out the dummy for the first year in the observation period, which serves as a baseline against which the effects of the other years are compared. 

In [43]:
# Create year dummies
year_dummies = pd.get_dummies(directors_years_control_variables['year'], prefix='year', drop_first=True)

# Concatenate the year dummies back to the original DataFrame
directors_years_control_variables = pd.concat([directors_years_control_variables, year_dummies], axis=1)

To include these dummies as a predictor in a statistical model, we need to convert them from boolean (True/False) to integer (1/0).

In [44]:
# Select columns that start with 'year_'
dummy_columns = [col for col in directors_years_control_variables.columns if col.startswith('year_')]

# Convert selected columns to integer
directors_years_control_variables[dummy_columns] = directors_years_control_variables[dummy_columns].astype(int)

In [45]:
# select the first two columns of the data along with the last 20 columns, and then selects the first 5 rows of the resulting DataFrame 
# to verify dummy variable creation
directors_years_control_variables.loc[:39, directors_years_control_variables.columns[:2].tolist() + directors_years_control_variables.columns[-20:].tolist()].head()

,nconst_director,year,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,...,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,nm1131265,2004,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,nm1131265,2005,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,nm1131265,2006,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,nm1131265,2007,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,nm1131265,2008,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 3.5. Directors' prior filmmaking experience 

In [46]:
directors_pre_debut_careers.head()

,nconst,had_prior_experience,movie_count,roles
0,nm1131265,0,0.0,[]
1,nm1130611,0,0.0,[]
2,nm0290651,0,0.0,[]
3,nm0404033,0,0.0,[]
4,nm0000417,1,14.0,"[archive_footage, actor]"


In [47]:
directors_years_control_variables = pd.merge(directors_years_control_variables,
                                             directors_pre_debut_careers[['nconst', 'had_prior_experience']].rename(columns={'nconst': 'nconst_director'}),
                                             on=['nconst_director'], how='left')

In [49]:
directors_years_control_variables.head()

,nconst_director,year,debut_year,director_total_movies,director_raverage_rating,director_total_votes,Action,Adventure,Animation,Biography,...,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023,had_prior_experience
0,nm1131265,2004,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,nm1131265,2005,2004,1.0,7.8,34.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,nm1131265,2006,2004,1.0,7.8,34.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,nm1131265,2007,2004,1.0,7.8,34.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,nm1131265,2008,2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


## 3.6. Directors' debut year

In [51]:
debut_year_dummies.head()

,nconst_director,debut_year_2004,debut_year_2005,debut_year_2006,debut_year_2007,debut_year_2008,debut_year_2009,debut_year_2010,debut_year_2011,debut_year_2012,debut_year_2013
0,nm1131265,True,False,False,False,False,False,False,False,False,False
1,nm1130611,True,False,False,False,False,False,False,False,False,False
2,nm0290651,False,False,False,False,False,False,False,False,False,False
3,nm0404033,False,False,False,False,False,False,False,False,False,False
4,nm0000417,False,True,False,False,False,False,False,False,False,False


In [52]:
# Select columns that start with 'debut_year_' or 'year_'
dummy_columns = [col for col in debut_year_dummies.columns if col.startswith('debut_year_')]

# Convert dummy columns to integer
debut_year_dummies[dummy_columns] = directors_cross_sectional_control_variables[dummy_columns].astype(int)

In [55]:
directors_years_control_variables = pd.merge(directors_years_control_variables,
                                             debut_year_dummies,
                                             on=['nconst_director'], how='left')

In [56]:
directors_years_control_variables.to_csv("directors_years_control_variables.csv", index = False)